In [6]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [1]:
from langchain.utilities import SQLDatabase

In [2]:
db = SQLDatabase.from_uri("sqlite:///./transcriptions.db")

In [3]:
def get_schema(_):
    return db.get_table_info()

In [4]:
def run_query(query):
    return db.run(query)

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

model = ChatOpenAI()

sql_response = (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        | model.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
    )

In [32]:
sql_response.invoke({"question": "read from all columns"})

'SELECT * FROM transcriptions'

In [33]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [34]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response) 
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response 
    | model
)

In [40]:
ans = full_chain.invoke({"question": "How many customers called on TV issue"})
import pprint
from pprint import pprint
pprint(ans)

AIMessage(content='There were 3 customers who called regarding a TV issue.')


In [42]:
ans = full_chain.invoke({"question": "Issue raised first today morning"})
import pprint
from pprint import pprint
pprint(ans)

AIMessage(content="The issue raised first today morning is the one with ID 1. The customer is asking about their subscription activation and confirming their number. The customer care representative is asking about the customer's number and confirming the WhatsApp payment.")
